In [2]:
import pandas as pd
import numpy as np
import json

In [28]:
# 제품 카테고리

path = '../csv/categories.csv'

df_categories = pd.read_csv(path)

df_categories['category_left_3'] = df_categories['category'].astype(str).str[:3]

df_categories.tail()

,category,name,short_name,parent_id,num_child,modified_at,img_url,img_url_w,cate1,cate2,...,path,depth,estimated_count,ad_enable,is_enable,checkout_enable,is_goods,category_name_indexable,sex,category_left_3
1219,240100030,서비스/미디어,NaN,240100.0,0.0,2018-02-26 00:00:00,NaN,NaN,NaN,NaN,...,/240/100/030,3,1,0,1,0,1,0,NaN,240
1220,240100050,생산/기능직,NaN,240100.0,0.0,2018-02-26 00:00:00,NaN,NaN,NaN,NaN,...,/240/100/050,3,1,0,1,0,1,0,NaN,240
1221,240100070,상담영업,NaN,240100.0,0.0,2018-02-26 00:00:00,NaN,NaN,NaN,NaN,...,/240/100/070,3,1,0,1,0,1,0,NaN,240
1222,240100090,기타 업종,NaN,240100.0,0.0,2018-02-26 00:00:00,NaN,NaN,NaN,NaN,...,/240/100/090,3,13,0,1,0,1,0,NaN,240
1223,240100110,단기알바,NaN,240100.0,0.0,2018-02-26 00:00:00,NaN,NaN,NaN,NaN,...,/240/100/110,3,9,0,1,0,1,0,NaN,240


### 검색 파일 핸들링

In [3]:
# 19년 4월 검색한 키워드의 상세 검색 활용 여부
path = '../csv/keyword_search.csv'

df_search = pd.read_csv(path)
df_search = df_search[df_search.viewer_uid > 0] # 비회원 제거

df_search.to_csv('../csv/tmp/df_search.csv', index=False, mode='w', header=True) # save

df_search.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail
22767271,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1
22767272,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1
22767273,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17
22767274,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3
22767275,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22


In [ ]:
# df_search_agg = df_search.groupby(['device']).agg({'count_search': 'sum', 'keyword': 'count'})


### 클릭 파일 핸들링

In [ ]:
# 19년 4월 클릭된 상품과 유입된 키워드

path = '../csv/keyword_click.csv'

df_click = pd.read_csv(path)
df_click = df_click[df_click.viewer_uid > 0] # 비회원 제거

df_click['category_left3'] = df_click['category'].astype(str).str[:3]

df_click.to_csv('../csv/tmp/df_click.csv', index=False, mode='w', header=True) # save

df_click.tail()

In [10]:
df_click = pd.read_csv('../csv/tmp/df_click.csv')

df_click.tail()

,viewer_uid,viewer_device,keyword,pid,category,name,count_click,category_left3
75633034,1501637,a,쇼바이크,81412202.0,750800100.0,오토바이(125cc 이하),1,750.0
75633035,7319500,w,응원봉 악세사리,101318080.0,910200007.0,기타(걸그룹),1,910.0
75633036,4225264,a,럽셀콘 dvd,99701886.0,910100001.0,팬시,1,910.0
75633037,4948901,a,에어팟,101408327.0,600500010.0,이어폰/헤드폰,1,600.0
75633038,4263581,a,팔라스 크루넥,100401393.0,320030200.0,후드 티셔츠,1,320.0


In [15]:

df_click_agg = df_click.groupby(['viewer_uid', 'viewer_device', 'keyword']).agg({'count_click': 'sum'})

df_click_agg.reset_index()

df_click_agg.to_csv('../csv/tmp/df_click_agg.csv', index=True, mode='w', header=True) # save

df_click_agg.tail()

count_click
viewer_uid viewer_device keyword             
8949058    a             지구젤리               9
8949063    a             블리치 55             1
8949064    a             아이폰8 깨짐            7
                         아이폰8 뒷면            2
8949074    a             네비게이션              1

In [17]:
df_click_agg = pd.read_csv('../csv/tmp/df_click_agg.csv')

df_click_agg.tail()

,viewer_uid,viewer_device,keyword,count_click
9346860,8949058,a,지구젤리,9
9346861,8949063,a,블리치 55,1
9346862,8949064,a,아이폰8 깨짐,7
9346863,8949064,a,아이폰8 뒷면,2
9346864,8949074,a,네비게이션,1


### 제품 검색에서 클릭으로의 전환

In [18]:
# Merge search with click 
df_search_click = pd.merge(df_search, df_click_agg, how='left', on=['viewer_uid', 'viewer_device','keyword'])

df_search_click.to_csv('../csv/tmp/df_search_click.csv', index=False, mode='w', header=True) # save

df_search_click.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click
21087268,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0
21087269,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0
21087270,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0
21087271,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0
21087272,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0


In [5]:
df_search_click = pd.read_csv('../csv/tmp/df_search_click.csv')

df_search_click.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click
21087268,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0
21087269,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0
21087270,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0
21087271,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0
21087272,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0


In [4]:
df_search_click['total_search'] = df_search_click['count_none'] + df_search_click['count_detail']

df_search_click.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click,total_search
21087268,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0,2
21087269,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0,2
21087270,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0,24
21087271,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0,6
21087272,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0,44


### 키워드의 카테고리

In [8]:
# 키워드 카테고리

path = '../csv/keyword_category.csv'

df_categories = pd.read_csv(path).drop(['_count', '_num'], axis=1)
df_categories = df_categories.rename(columns={'categroy': 'category_keyword', 'name': 'category_name_keyword'})

df_categories.tail()

,keyword,category,category_name_keyword
1891622,💖신상🌸,320,남성의류
1891623,💬찢청,320,남성의류
1891624,💯발렌시아가 빌 탑,400,패션잡화
1891625,🚛정품 MCM,400,패션잡화
1891626,🤘구찌 지갑,400,패션잡화


In [15]:
# 키워드 세부 카테고리

path = '../csv/keyword_category_detail.csv'

df_categories_detail = pd.read_csv(path).drop(['_count', '_num'], axis=1)
df_categories_detail = df_categories_detail.rename(columns={'categroy': 'category_keyword', 'name': 'category_name_keyword'})

df_categories_detail.tail()

,keyword,category,category_name_keyword
1891622,💖신상🌸,320030100.0,맨투맨 티셔츠
1891623,💬찢청,320130200.0,청/데님 반바지
1891624,💯발렌시아가 빌 탑,400010100.0,숄더백
1891625,🚛정품 MCM,400083200.0,남성 벨트
1891626,🤘구찌 지갑,400060400.0,남자 중/반지갑


### 키워드에 카테고리 붙이기

In [9]:
df_search_click_category = pd.merge(df_search_click, df_categories, how = 'left', on = 'keyword')

df_search_click_category.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click,category,category_name_keyword
21087268,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0,600.0,디지털/가전
21087269,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0,600.0,디지털/가전
21087270,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0,400.0,패션잡화
21087271,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0,700.0,스포츠/레저
21087272,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0,310.0,여성의류


In [36]:
df_search_click_cate_agg = df_search_click_category.groupby(['viewer_device', 
                                                             'category_name_keyword']).agg('sum')

df_search_click_cate_agg.to_csv('../csv/tmp/df_search_click_cate_agg.csv', index=True, mode='w', header=True) # save

df_search_click_cate_agg

viewer_uid  count_location  \
viewer_device category_name_keyword                                  
a             (구)패션의류                     28464191               2   
              구인구직                     14413359203            5146   
              기타                      912628892861          387979   
              남성의류                   7358335097170         4584732   
              도서/티켓/취미/애완            3446274432513         1933042   
              디지털/가전                 9355271668754         5194089   
              번개나눔                     53644409426           28223   
              뷰티/미용                  1592912061258          678831   
              생활/문구/가구/식품            2458210831737         1253471   
              스타굿즈                   1780295093507         1008650   
              스포츠/레저                 3846352436178         2345016   
              여성의류                   2974592175158         2246209   
              원룸/함께살아요                 15688529837            6211   
              유아동/출산                  872158315542          503619   
              재능                       33471837461           10627   
              지역 서비스                    7594964679            2350   
              차량/오토바이                2731055008423         1650537   
              커뮤니티                     17869133009            3563   
              패션잡화                   9239120433852         6099174   
i             (구)패션의류                      3572504               1   
              구인구직                      7921006779            2095   
              기타                      560911512494          181282   
              남성의류                   7607100504762         2426908   
              도서/티켓/취미/애완            1871071409496          622098   
              디지털/가전                 6076996995375         2285553   
              번개나눔                     25883605737            8275   
              뷰티/미용                  1277502057914          363311   
              생활/문구/가구/식품            1249379246268          366560   
              스타굿즈                   1184601419775          416301   
              스포츠/레저                 1729491724858          626210   
              여성의류                   3036188196442          981575   
              원룸/함께살아요                  5220968590            1351   
              유아동/출산                  369186294344          119125   
              재능                       30389510140            9352   
              지역 서비스                    3624820026            1014   
              차량/오토바이                1242969947759          464923   
              커뮤니티                     10172338709            1915   
              패션잡화                   8966281861677         2850694   
w             구인구직                      1372568285               0   
              기타                       89508551731               0   
              남성의류                    674093560414               0   
              도서/티켓/취미/애완             339367064765               0   
              디지털/가전                 1164874143522               0   
              번개나눔                      3187889965               0   
              뷰티/미용                    76655953858               0   
              생활/문구/가구/식품             157269885367               0   
              스타굿즈                    160543021714               0   
              스포츠/레저                  317249849010               0   
              여성의류                    193834547084               0   
              원룸/함께살아요                  1758260066               0   
              유아동/출산                   29602799795               0   
              재능                        4909619055               0   
              지역 서비스                     815979214               0   
              차량/오토바이                 212500736328               0   
              커뮤니티                      2001859237               0   
              패션잡화                    729894294842

In [7]:
# 세부 카테고리 붙이기

df_search_click_cate_detail = pd.merge(df_search_click, df_categories_detail, how = 'left', on = 'keyword')

df_search_click_cate_detail.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click,total_search,category,category_name_keyword
21087268,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0,2,600700007.0,배터리/충전기
21087269,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0,2,600500010.0,이어폰/헤드폰
21087270,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0,24,400040100.0,런닝화/워킹화
21087271,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0,6,700350950.0,자전거 부품
21087272,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0,44,310100010.0,기본/테일러드 자켓


In [8]:
df_search_click_cate_detail_agg = df_search_click_cate_detail.groupby(['viewer_device',
                                                                       'category',
                                                                       'category_name_keyword']).agg('sum')

df_search_click_cate_detail_agg.to_csv('../csv/tmp/df_search_click_cate_detail_agg.csv', index=True, mode='w', header=True) # save

df_search_click_cate_detail_agg

viewer_uid  \
viewer_device category    category_name_keyword                  
a             210.0       재능                           4435359   
              310.0       여성의류                       361832689   
              320.0       남성의류                       318217572   
              400.0       패션잡화                       274483424   
              410.0       뷰티/미용                      712730930   
              500.0       유아동/출산                     172894792   
              600.0       디지털/가전                     339816829   
              700.0       스포츠/레저                     244400022   
              750.0       차량/오토바이                    252257911   
              800.0       생활/문구/가구/식품                453548840   
              900.0       도서/티켓/취미/애완                148471153   
              910.0       스타굿즈                       138811615   
              999.0       기타                     1336196657020   
              100200.0    수다방                       1757877969   
              100400.0    칭찬해요(후기)                  1067342146   
              100600.0    주의해요(후기)                 12484799887   
              100800.0    광고방/상점공지                  3767641989   
              200100.0    물품 무료나눔                  51784582061   
              200200.0    먹거리 무료나눔                   862736778   
              200300.0    덤(조건부나눔)                  1552962396   
              200400.0    당첨발표(당발)                  1101453897   
              200999.0    기타                        2060868206   
              210211.0    스타일/뷰티                       1550495   
              220010.0    네일/미용                      444849485   
              220020.0    호텔/펜션/숙박                  2082710181   
              220030.0    학원/수강                      151316329   
              220040.0    인테리어/간판                    517759239   
              220050.0    헬스/요가                      864989860   
              220060.0    병원/약국                       21862288   
              220070.0    차량/수리                      118992835   
...                                                        ...   
w             910400005.0 패션의류                        28595048   
              910400006.0 패션잡화                        12508765   
              910400007.0 기타(솔로-여)                   802991361   
              910500001.0 팬시                          42819234   
              910500002.0 응원도구                        51150775   
              910500003.0 포스터/화보                      54244320   
              910500004.0 음반/영상물                     800337020   
              910500005.0 패션의류                        17386899   
              910500006.0 패션잡화                        57382083   
              910500007.0 기타(솔로-남)                   135028585   
              910500008.0 인형/피규어                      31538339   
              910600001.0 팬시                         132643456   
              910600002.0 응원도구                         8662453   
              910600003.0 포스터/화보                     322311842   
              910600004.0 음반/영상물                     116558829   
              910600005.0 패션의류                         7783497   
              910600006.0 패션잡화                         4291524   
              910600007.0 기타(배우-남)                   256209962   
              910600008.0 인형/피규어                      50383401   
              910700003.0 포스터/화보                      82174325   
              910700005.0 패션의류                         5911486   
              910700007.0 기타(배우-여)                    99813033   
              910800001.0 팬시                         125370654   
              910800002.0 응원도구                        11660467   
              910800003.0 포스터/화보                      23275768   
              910800004.0 음반/영상물                      71685767   
              910800005.0 패션의류                        70758176   
              910800006.0 패션잡화                         3084691   
              910800007.0 

### 검색 유저의 정보

In [12]:
df_search_user = pd.read_csv('../csv/keyword_search_user.csv')

df_search_user = df_search_user[df_search_user.viewer_uid > 0] # 비회원 제거

df_search_user.to_csv('../csv/tmp/df_search_user.csv', index=False, mode='w', header=True) # save

df_search_user.tail()

,viewer_uid,device,age,gender,bizlicense
7812067,8949037,i,22.0,0.0,0.0
7812068,8949055,a,0.0,-1.0,0.0
7812069,8949063,w,0.0,-1.0,0.0
7812070,8949073,a,0.0,-1.0,0.0
7812071,8949075,a,0.0,-1.0,0.0


In [3]:
df_search_user = pd.read_csv('../csv/tmp/df_search_user.csv')

df_search_user.tail()

,viewer_uid,device,age,gender,bizlicense
7812038,8949037,i,22.0,0.0,0.0
7812039,8949055,a,0.0,-1.0,0.0
7812040,8949063,w,0.0,-1.0,0.0
7812041,8949073,a,0.0,-1.0,0.0
7812042,8949075,a,0.0,-1.0,0.0


In [20]:
df_search_user = df_search_user.drop_duplicates()

df_search_user.tail()

,viewer_uid,device,age,gender,bizlicense
7812038,8949037,i,22.0,0.0,0.0
7812039,8949055,a,0.0,-1.0,0.0
7812040,8949063,w,0.0,-1.0,0.0
7812041,8949073,a,0.0,-1.0,0.0
7812042,8949075,a,0.0,-1.0,0.0


In [10]:
df_search_click_category.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click,category,category_name_keyword
21087268,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0,600.0,디지털/가전
21087269,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0,600.0,디지털/가전
21087270,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0,400.0,패션잡화
21087271,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0,700.0,스포츠/레저
21087272,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0,310.0,여성의류


In [21]:
df_search_click_category_user = pd.merge(df_search_click_category, df_search_user, how = 'left', on = 'viewer_uid')

df_search_click_category_user.to_csv('../csv/tmp/df_search_click_category_user.csv', index=False, mode='w', header=True) # save

df_search_click_category_user.tail()

,viewer_uid,viewer_device,keyword,count_location,count_category,count_price_min,count_price_max,count_used,count_bizseller,count_free_shipping,count_exchg,count_none,count_detail,count_click,category,category_name_keyword,device,age,gender,bizlicense
21087338,7785690,i,무선충전기,1,0,0,0,0,0,0,0,1,1,1.0,600.0,디지털/가전,i,25.0,1.0,0.0
21087339,1814114,i,에어팟,1,0,0,0,0,0,0,0,1,1,1.0,600.0,디지털/가전,i,24.0,1.0,0.0
21087340,1849495,a,의정부 송산,17,0,0,0,0,0,0,0,7,17,10.0,400.0,패션잡화,a,0.0,-1.0,0.0
21087341,8535956,a,3t에어로노바,3,0,0,0,0,0,0,0,3,3,5.0,700.0,스포츠/레저,w,0.0,-1.0,0.0
21087342,8212458,i,데쥬,22,0,0,0,0,0,0,0,22,22,40.0,310.0,여성의류,w,19.0,0.0,0.0


In [22]:
df_search_click_category_user_agg = df_search_click_category_user.groupby(['viewer_device', 'age', 'gender',
                                                                       'category',
                                                                       'category_name_keyword']).agg('sum')

df_search_click_category_user_agg.to_csv('../csv/tmp/df_search_click_category_user_agg.csv', index=True, mode='w', header=True) # save

df_search_click_category_user_agg

viewer_uid  \
viewer_device age   gender category category_name_keyword                  
a             0.0   -1.0   100.0    커뮤니티                      7400466152   
                           200.0    번개나눔                     26381165335   
                           210.0    재능                       11071488606   
                           220.0    지역 서비스                    2626138692   
                           230.0    원룸/함께살아요                  6547232185   
                           240.0    구인구직                      4690041730   
                           300.0    (구)패션의류                     28464191   
                           310.0    여성의류                   1079137986213   
                           320.0    남성의류                   2648183682592   
                           400.0    패션잡화                   3469154209443   
                           410.0    뷰티/미용                   546577827655   
                           500.0    유아동/출산                  322265520931   
                           600.0    디지털/가전                 4026313973036   
                           700.0    스포츠/레저                 1940592742342   
                           750.0    차량/오토바이                1128316373250   
                           800.0    생활/문구/가구/식품            1038360918002   
                           900.0    도서/티켓/취미/애완            1382168608821   
                           910.0    스타굿즈                    823504720027   
                           999.0    기타                      378336083603   
              14.0   0.0   100.0    커뮤니티                        15625664   
                           200.0    번개나눔                       220204491   
                           210.0    재능                          27564673   
                           230.0    원룸/함께살아요                     8039241   
                           240.0    구인구직                        43437609   
                           310.0    여성의류                      3946617973   
                           320.0    남성의류                      3277850503   
                           400.0    패션잡화                      3907874360   
                           410.0    뷰티/미용                     4091491616   
                           500.0    유아동/출산                     166858329   
                           600.0    디지털/가전                    4488692951   
...                                                                  ...   
w             86.0   1.0   320.0    남성의류                        13592122   
                           600.0    디지털/가전                       4060016   
                           700.0    스포츠/레저                       8120032   
              87.0   0.0   320.0    남성의류                        68112840   
                           600.0    디지털/가전                      22729804   
                           750.0    차량/오토바이                     27670356   
                           800.0    생활/문구/가구/식품                 11352140   
                     1.0   320.0    남성의류                         3808665   
                           900.0    도서/티켓/취미/애완                  3808665   
                           999.0    기타                           3808665   
              88.0   0.0   310.0    여성의류                         7100684   
                           320.0    남성의류                        10651026   
                           400.0    패션잡화                         1808383   
                           600.0    디지털/가전                      15936069   
                           700.0    스포츠/레저                      11672326   
                     1.0   400.0    패션잡화                         7358637   
              89.0   0.0   320.0    남성의류                         1685811   
                           400.0    패션잡화                        16079496   
                           910.0    스타굿즈                        13128906   
                     1.0   210.0    재능                           5619731   
                           400.0  